# Energy Measurement Evaluation

This notebook evaluates power consumption and energy trends from experiment results.  
The data is collected from multiple nodes and analyzed for insights into power usage, voltage, and energy consumption.

In [ ]:
import os
from IPython.display import display, HTML
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Specify the Result Folder

Before loading data, enter the path to your experiment result folder.  
By default, the last used path is shown, but you can change it to any valid directory.

In [ ]:
base_result_folder = "/srv/testbed/results/warmuth/default/"
default_result_folder = os.path.join(base_result_folder, "2025-03-19_11-59-58_754222")

# -----------------------------------
# STEP 1: Select Result Folder
# -----------------------------------
user_input = input(
    f"\nEnter result folder path (leave empty to use default):\n[{default_result_folder}]\n> "
).strip()

# Resolve path
if not user_input:
    RESULT_FOLDER = default_result_folder
elif os.path.isabs(user_input):
    RESULT_FOLDER = user_input
else:
    RESULT_FOLDER = os.path.join(base_result_folder, user_input)

# Validate result folder
if not os.path.exists(RESULT_FOLDER):
    raise FileNotFoundError(f"\n[Error] Result folder does not exist:\n{RESULT_FOLDER}")

print(f"\n[Info] Using result folder:\n{RESULT_FOLDER}")

# -----------------------------------
# STEP 2: Detect Available Runs
# -----------------------------------
energy_folder = os.path.join(RESULT_FOLDER, "energy")
available_runs = set()

if os.path.exists(energy_folder):
    for node in os.listdir(energy_folder):
        node_path = os.path.join(energy_folder, node)
        if os.path.isdir(node_path):
            for filename in os.listdir(node_path):
                if filename.endswith(".csv") and "_run" in filename:
                    run_id = filename.split("_run")[-1].split(".")[0]
                    available_runs.add(run_id)

available_runs = sorted(available_runs)

if not available_runs:
    raise ValueError(f"\n[Warning] No runs found in folder:\n{energy_folder}")

print(f"\n[Info] Detected {len(available_runs)} run(s):\n{', '.join(available_runs)}")

# -----------------------------------
# STEP 3: Select Runs to Load
# -----------------------------------
run_input = input("\nEnter run numbers to include (comma-separated), or press Enter to include all:\n> ").strip()

if run_input:
    selected_runs = {run.strip() for run in run_input.split(",")}
    invalid_runs = selected_runs - set(available_runs)

    if invalid_runs:
        raise ValueError(
            f"\n[Error] Invalid run(s) entered:\n{', '.join(sorted(invalid_runs))}"
            f"\nAvailable runs:\n{', '.join(available_runs)}"
        )

    print(f"\n[Info] Filtering for {len(selected_runs)} run(s):\n{', '.join(sorted(selected_runs))}")
else:
    selected_runs = None
    print("\n[Info] Including all available runs.")


# -----------------------------------
# STEP 4: Detect and Select Nodes
# -----------------------------------
available_nodes = sorted([
    node for node in os.listdir(energy_folder)
    if os.path.isdir(os.path.join(energy_folder, node))
])

if not available_nodes:
    raise ValueError(f"\n[Warning] No node directories found in:\n{energy_folder}")

print(f"\n[Info] Detected {len(available_nodes)} node(s):\n{', '.join(available_nodes)}")

node_input = input("\nEnter node names to include (comma-separated),\n or press Enter to include all:\n> ").strip()

if node_input:
    selected_nodes = {n.strip() for n in node_input.split(",")}
    invalid_nodes = selected_nodes - set(available_nodes)

    if invalid_nodes:
        raise ValueError(
            f"\n[Error] Invalid node(s) entered:\n{', '.join(sorted(invalid_nodes))}"
            f"\nAvailable nodes:\n{', '.join(available_nodes)}"
        )

    print(f"\n[Info] Filtering for {len(selected_nodes)} node(s):\n{', '.join(sorted(selected_nodes))}")
else:
    selected_nodes = None
    print("\n[Info] Including all available nodes.")


## Creator Information

The following table presents details about the experiment's creator, extracted from the **RO-Crate metadata**.

- **Name:** The name of the creator.
- **ORCID:** A unique researcher identifier, linked to the official ORCID profile.
- **Affiliation:** The institution the creator is affiliated with.
- **Affiliation ROR:** A **Research Organization Registry (ROR) ID**, used for standard identification of research institutions.
- **Affiliation URL:** A direct link to the institution’s website.

In [ ]:
def load_creator_info():
    """
    Extracts all creators from the RO-Crate metadata JSON file.
    Retrieves each creator's name, ORCID, and affiliation details.
    """
    rocrate_path = os.path.join(RESULT_FOLDER, "ro-crate-metadata.json")
    if not os.path.exists(rocrate_path):
        raise FileNotFoundError(f"RO-Crate metadata file not found: {rocrate_path}")

    with open(rocrate_path, "r") as f:
        metadata = json.load(f)

    creators = []
    credit_url = "https://www.elsevier.com/researcher/author/policies-and-guidelines/credit-author-statement"

    for item in metadata.get("@graph", []):
        if item.get("@type") == "Person" and "creator" in item.get("tags", []):
            creator_info = {
                "Creator Name": item.get("name", "Unknown"),
                "ORCID": item.get("@id", "Unknown"),
                "Contribution (CRediT)": (
                    f'<a href={credit_url} target="_blank">{item.get("description", "Not specified")}</a>'
                ),
                "Affiliation Name": "Unknown",
                "Affiliation ROR": "Unknown",
                "Affiliation URL": "Unknown"
            }

            # Find affiliation
            affiliation_id = item.get("affiliation", {}).get("@id", None)
            if affiliation_id:
                for org in metadata.get("@graph", []):
                    if org.get("@id") == affiliation_id:
                        creator_info["Affiliation Name"] = org.get("name", "Unknown")
                        creator_info["Affiliation ROR"] = org.get("@id", "Unknown")
                        creator_info["Affiliation URL"] = org.get("url", "Unknown")
                        break

            creators.append(creator_info)

    return creators

creator_data = load_creator_info()
creator_df = pd.DataFrame(creator_data)

def make_link(text, url):
    return f'<a href="{url}" target="_blank">{text}</a>' if url != "Unknown" else "Unknown"

creator_df["ORCID"] = creator_df["ORCID"].apply(lambda x: make_link("ORCID Profile", x))
creator_df["Affiliation ROR"] = creator_df["Affiliation ROR"].apply(lambda x: make_link("ROR ID", x))
creator_df["Affiliation URL"] = creator_df["Affiliation URL"].apply(lambda x: make_link("University Website", x))

html_table = creator_df.to_html(escape=False, index=False)
styled_table = f"""
<style>
    table {{ width: 80%; border-collapse: collapse; margin: 20px 0; }}
    th, td {{ padding: 8px 12px; border: 1px solid #ddd; text-align: left; }}
    th {{ background-color: #f4f4f4; font-weight: bold; }}
</style>
{html_table}
"""

display(HTML(styled_table))

## Node Information & Topology Visualization

Each experiment setup includes metadata about the participating nodes.  
This section extracts details such as:
- Node names
- Links to the Testbed -> Entrypoint to Testbed
- Fully Qualified Domain Names (FQDN)
- Topology information (if available).

If a **topology visualization** is provided in the RO-Crate metadata, it is linked below.

In [ ]:
def load_rocrate_metadata():
    """
    Load and parse the RO-Crate metadata JSON file.
    Extract node information and locate paths for hardware details and topology PDFs.
    """
    rocrate_path = os.path.join(RESULT_FOLDER, "ro-crate-metadata.json")
    if not os.path.exists(rocrate_path):
        raise FileNotFoundError(f"RO-Crate metadata file not found: {rocrate_path}")

    with open(rocrate_path, "r") as f:
        metadata = json.load(f)

    nodes_info = []

    for item in metadata.get("@graph", []):
        if "tags" in item and "node" in item["tags"]:
            node_name = item.get("name", "Unknown")
            fqdn = item.get("fqdn", "Unknown")

            topology_pdf_path = None
            if isinstance(item.get("visualizedTopology", {}), dict) and "@id" in item["visualizedTopology"]:
                topology_pdf_path = os.path.join(RESULT_FOLDER, item["visualizedTopology"]["@id"])
                print(topology_pdf_path)
                if not os.path.exists(topology_pdf_path):
                    topology_pdf_path = None

            hardware_json_path = None
            if isinstance(item.get("hardware", {}), dict) and "@id" in item["hardware"]:
                hardware_json_path = os.path.join(RESULT_FOLDER, item["hardware"]["@id"])
                print(hardware_json_path)
                if not os.path.exists(hardware_json_path):
                    hardware_json_path = None

            nodes_info.append({
                "name": node_name if isinstance(node_name, str) else "Unknown",
                "fqdn": fqdn if isinstance(fqdn, str) else "Unknown",
                "topology_pdf": topology_pdf_path if topology_pdf_path else "None",
                "hardware_json": hardware_json_path if hardware_json_path else "None"
            })

    return nodes_info

def extract_hardware_info(hardware_json_path):
    """
    Extract CPU, memory, and NIC info from the given hardware.json file.
    Handles multiple CPUs and formats output for display. Robust to missing or None values.
    """
    if not hardware_json_path or not os.path.exists(hardware_json_path):
        return {
            "cpu_model": "Unknown", "cpu_cores": "Unknown", "cpu_threads": "Unknown",
            "memory": "Unknown", "nic_models": "Unknown"
        }

    try:
        with open(hardware_json_path, "r") as f:
            hardware_data = json.load(f)

        # CPU(s)
        cpus = hardware_data.get("processor", [])
        cpu_models = [str(cpu.get("model") or "Unknown") for cpu in cpus]
        cpu_cores = [str(cpu.get("cores") or "Unknown") for cpu in cpus]
        cpu_threads = [str(cpu.get("threads") or "Unknown") for cpu in cpus]

        cpu_model_str = ", ".join(cpu_models)
        cpu_cores_str = ", ".join(cpu_cores)
        cpu_threads_str = ", ".join(cpu_threads)

        # Memory
        memory_info = hardware_data.get("memory", {})
        mem_val = memory_info.get("installed_capacity_human_val", "Unknown")
        mem_unit = memory_info.get("installed_capacity_human_unit", "")
        memory_str = f"RAM: {mem_val} {mem_unit}".strip() if mem_val != "Unknown" else "Unknown"

        # NICs
        nic_models = []
        for nic in hardware_data.get("network", []):
            if isinstance(nic, dict):
                nic_model = nic.get("model")
                if nic_model:
                    nic_models.append(str(nic_model))
        nic_str = "<br>".join(nic_models) if nic_models else "No NICs detected"

        return {
            "cpu_model": cpu_model_str,
            "cpu_cores": cpu_cores_str,
            "cpu_threads": cpu_threads_str,
            "memory": memory_str,
            "nic_models": nic_str
        }

    except Exception as e:
        print(f"[ERROR] Failed to parse hardware.json: {e}")
        return {
            "cpu_model": "Unknown", "cpu_cores": "Unknown", "cpu_threads": "Unknown",
            "memory": "Unknown", "nic_models": "Unknown"
        }



# Load node metadata and hardware details
nodes_info = load_rocrate_metadata()
nodes_df = pd.DataFrame(nodes_info)
hardware_details = [extract_hardware_info(node["hardware_json"]) for node in nodes_info]
hardware_df = pd.DataFrame(hardware_details)
nodes_df = pd.concat([nodes_df, hardware_df], axis=1)
nodes_df.drop(columns=["hardware_json"], inplace=True)

def extract_testbed(fqdn):
    parts = fqdn.split(".")
    if len(parts) > 1:
        return parts[1].capitalize()

nodes_df["Testbed Entrypoint"] = nodes_df["fqdn"].apply(extract_testbed)

testbed_urls = {
    "Baltikum": "https://kaunas.net.cit.tum.de/",
    "Blockchain": "https://coinbase.net.cit.tum.de/"
}

def make_testbed_link(testbed):
    url = testbed_urls.get(testbed, "Unknown")
    return f'<a href="{url}" target="_blank">{testbed}</a>' if url != "Unknown" else "Unknown"

nodes_df["Testbed Entrypoint"] = nodes_df["Testbed Entrypoint"].apply(make_testbed_link)

def make_clickable(path):
    return f'<a href="{path}" target="_blank">Open PDF</a>' if path != "None" else "No topology available"

nodes_df["topology_pdf"] = nodes_df["topology_pdf"].apply(make_clickable)

# Rename columns for better readability
nodes_df.rename(columns={
    "name": "Name",
    "fqdn": "FQDN",
    "topology_pdf": "Topology",
    "cpu_model": "CPU",
    "cpu_cores": "Cores",
    "cpu_threads": "Threads",
    "memory": "Memory",
    "nic_models": "NICs",
    "Testbed": "Testbed"
}, inplace=True)

nodes_df = nodes_df[["Name", "FQDN", "Testbed Entrypoint", "Topology", "CPU", "Cores", "Threads", "Memory", "NICs"]]

html_table = nodes_df.to_html(escape=False)
styled_table = f"""
<style>
    table {{ width: 90%; border-collapse: collapse; margin: 20px 0; }}
    th, td {{ padding: 8px 12px; border: 1px solid #ddd; text-align: left; }}
    th {{ background-color: #f4f4f4; font-weight: bold; }}
</style>
{html_table}
"""

display(HTML(styled_table))

## Loading and Previewing Data

The energy measurement data is stored in CSV format, with each node having its own folder inside the `energy` directory.

The dataset includes:
- **Timestamp** (`timestamp`): Time when the measurement was recorded.
- **Current** (`current_mA`): Measured current in milliamps (mA).
- **Voltage** (`voltage_V`): Measured voltage in volts (V).
- **Power Consumption** (`power_active_W`): Active power in watts (W).
- **Energy Counter** (`energy_counter_Wh`): Cumulative energy usage in watt-hours (Wh).

Below, we load the data and display a preview.

In [ ]:
sns.set_theme(style="whitegrid")
plt.rcParams.update({"axes.titlesize": 14, "axes.labelsize": 12})

def load_energy_data(selected_runs=None):
    """
    Load and merge energy measurement data from CSV files.
    If values are split into *_0, *_1 (e.g., current_mA_0, current_mA_1), they are summed into a unified column.
    """
    energy_folder = os.path.join(RESULT_FOLDER, "energy")
    if not os.path.exists(energy_folder):
        raise FileNotFoundError(f"Energy folder not found: {energy_folder}")

    all_data = []

    for node in os.listdir(energy_folder):
        if selected_nodes and node not in selected_nodes:
            continue
        node_path = os.path.join(energy_folder, node)
        if os.path.isdir(node_path):
            for file in os.listdir(node_path):
                if file.endswith(".csv") and "_run" in file:
                    run_id = file.split("_run")[-1].split(".")[0]
                    if selected_runs is not None and run_id not in selected_runs:
                        continue
                    file_path = os.path.join(node_path, file)
                    print(file_path)

                    df = pd.read_csv(file_path)
                    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y%m%d%H%M%S%f")
                    df["node"] = node
                    df["run"] = run_id

                    # Auto-detect and merge *_0 + *_1 columns
                    cols = df.columns
                    metric_prefixes = set()
                    for col in cols:
                        if col.endswith("_0") or col.endswith("_1"):
                            metric_prefixes.add(col.rsplit("_", 1)[0])

                    for prefix in metric_prefixes:
                        col_0 = f"{prefix}_0"
                        col_1 = f"{prefix}_1"
                        if col_0 in df.columns and col_1 in df.columns:
                            df[prefix] = df[col_0] + df[col_1]
                            df.drop([col_0, col_1], axis=1, inplace=True)

                    all_data.append(df)

    if not all_data:
        raise ValueError("No valid CSV files found in the energy folder.")

    return pd.concat(all_data, ignore_index=True)


df = load_energy_data(selected_runs)

def remove_outliers(df):
    """
    Removes extreme outliers from all numeric columns using the IQR method.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

df = remove_outliers(df)
display(df.head())
display(df.tail())

## Common Set of Statistical Evaluations

After loading the data, the notebook performs a standard set of statistical evaluations  
to understand the structure and integrity of the dataset. This includes:

- Summary statistics of numerical columns  
- Detection of missing values  
- Identification of outliers or unusual value ranges  
- Checks for consistency across runs and nodes  

These steps support further analysis by providing insights into data quality and distribution.

In [ ]:
temp_df = df.drop(columns=['node', 'run'])
display(temp_df.describe(exclude=[np.datetime64]))

## Cumulative Energy Consumption

The `energy_counter_mWh` tracks the cumulative energy consumed over time, converted from watt-hours to milliwatt-hours (mWh).  
This line plot shows how each node's energy consumption grows throughout the experiment, offering a clear comparison of power usage trends between nodes.


In [ ]:
# ---------------------------------------------------------------------
# Total Corrected Energy Consumption Per Node
# - Aggregates total energy used per node, per run.
# - Handles resets: if a counter drops mid-run, uses last value only.
# ---------------------------------------------------------------------

def compute_corrected_energy(df):
    """
    Computes total corrected energy per node/run, accounting for resets.
    """
    corrected_energy = []

    for (node, run), group in df.groupby(["node", "run"]):
        first_value = group["energy_counter_Wh"].iloc[0]
        last_value = group["energy_counter_Wh"].iloc[-1]

        if (group["energy_counter_Wh"].diff() < 0).any():
            energy_used = last_value  # Reset occurred-> use only last
        else:
            energy_used = last_value - first_value

        corrected_energy.append({
            "node": node,
            "run": run,
            "energy_used": energy_used
        })

    return pd.DataFrame(corrected_energy)

# Prepare data
df_corrected = compute_corrected_energy(df)
df_grouped = df_corrected.groupby("node")["energy_used"].sum().reset_index()

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Use seaborn barplot with automatic color assignment
sns.barplot(
    data=df_grouped,
    x="node",
    y="energy_used",
    hue="node",
    palette="pastel",
    dodge=False,
    legend=False,
    width=0.5  # thinner bars
)


# Add value labels on top of bars
for index, row in df_grouped.iterrows():
    plt.text(index, row.energy_used + 0.01, f"{row.energy_used:.2f}", ha="center", va="bottom", fontsize=10)

plt.ylabel("Total Corrected Energy (Wh)")
plt.xlabel("Node")
plt.title("Total Corrected Energy Consumption Per Node")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Power Consumption Over Time

The following plot shows the power consumption trends over time for different nodes.  
This helps us observe variations in power usage and detect potential anomalies.

In [ ]:
# ------------------------------------------------------------
# Plots Instantaneous Power Consumption (in Watts) over time
# (uses absolute timestamps on the x-axis)
# ------------------------------------------------------------

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="timestamp", y="power_active_W", hue="node", linewidth=2)
plt.xlabel("Timestamp")
plt.ylabel("Power Consumption (W)")
plt.title("Power Consumption Over Time")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


# ------------------------------------------------------------------------
# Plots Instantaneous Power Consumption (in Watts) over relative time
# (x-axis is seconds since the experiment started)
# ------------------------------------------------------------------------

df["timestamp_relative"] = (df["timestamp"] - df["timestamp"].min()).dt.total_seconds()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="timestamp_relative", y="power_active_W", hue="node", linewidth=2)
plt.xlabel("Time (s)")
plt.ylabel("Power Consumption (W)")
plt.title("Power Consumption Over Time (Relative)")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


## Cumulative Energy Consumption

The energy counter represents the cumulative energy consumed over time.  
This plot provides insights into the total energy usage per node and how it changes over the experiment duration.

In [ ]:
# -----------------------------------------------
# Plots Cumulative Energy Consumption in mWh
# (merged over all runs, grouped by node)
# -----------------------------------------------

df["energy_counter_mWh"] = df["energy_counter_Wh"] * 1000

plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df,
    x="timestamp",
    y="energy_counter_mWh",
    hue="node",
    linewidth=2
)

plt.xlabel("Timestamp")
plt.ylabel("Cumulative Energy (mWh)")
plt.title("Cumulative Energy Consumption Over Time")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


## Current and Voltage Trends

To better understand the electrical characteristics, we visualize:
- **Current (mA) over time** to see how power draw fluctuates.
- **Voltage (V) over time** to ensure stability across measurements.

In [ ]:
# ------------------------------------------------------------------
# Plots Current (mA) over Time
# ------------------------------------------------------------------

first_timestamps = df.groupby("run")["timestamp"].min()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="timestamp", y="current_mA", hue="node", linewidth=2)

plt.xlabel("Timestamp")
plt.ylabel("Current (mA)")
plt.title("Current Trend Over Time")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')

# Mark run start times
# ylim = plt.ylim()
# text_ypos = ylim[0] - (ylim[1] - ylim[0]) * 0.03  # 3% below plot bottom

# for run, ts in first_timestamps.items():
#     plt.axvline(x=ts, color="black", linestyle="dashed", alpha=0.4)
#     plt.text(ts, text_ypos, f"Run {run}", rotation=90, fontsize=9, color="black",
#              verticalalignment="top", horizontalalignment="center")

plt.tight_layout()
plt.show()


# ---------------------------------------------------------------------------
# Plots Smoothed Voltage (V) over Time
# Rolling mean is applied over a window of 5 samples
# ---------------------------------------------------------------------------

df["voltage_V_smoothed"] = df["voltage_V"].rolling(window=5, min_periods=1).mean()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="timestamp", y="voltage_V_smoothed", hue="node", linewidth=2)

plt.xlabel("Timestamp")
plt.ylabel("Voltage (V)")
plt.title("Voltage Trend Over Time (Smoothed)")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')

# Mark run start times
# ylim = plt.ylim()
# text_ypos = ylim[0] - (ylim[1] - ylim[0]) * 0.03

# for run, ts in first_timestamps.items():
#     plt.axvline(x=ts, color="black", linestyle="dashed", alpha=0.4)
#     plt.text(ts, text_ypos, f"Run {run}", rotation=90, fontsize=9, color="black",
#              verticalalignment="top", horizontalalignment="center")

plt.tight_layout()
plt.show()


### Energy Consumption Rate Over Time

This plot shows the **rate at which energy is consumed over time (mW/s)**.  
Instead of cumulative energy, this visualization helps identify **periods of high workload**.  
A higher energy rate means that the system was **actively consuming more power**,  
which may indicate high CPU load or network traffic.

In [ ]:
# ---------------------------------------------------------------------
# Raw Energy Counter Over Time Plot
# - Shows raw, unprocessed energy counter values in Wh.
# - Useful for identifying resets or measurement inconsistencies.
# ---------------------------------------------------------------------

plt.figure(figsize=(12, 6))

for node, group in df.groupby("node"):
    plt.plot(group["timestamp"], group["energy_counter_Wh"], label=node, marker="o", linestyle="-")

plt.xlabel("Timestamp")
plt.ylabel("Energy Counter (Wh)")
plt.title("Energy Counter Over Time (Raw Data)")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


# ---------------------------------------------------------------------
# Energy Consumption Rate (mW/s) - Corrected
# - Computes instantaneous energy rate from counter differences.
# - Handles counter resets by forward-filling previous values.
# ---------------------------------------------------------------------

df["energy_diff"] = df["energy_counter_Wh"].diff()
df["energy_adjusted"] = df["energy_counter_Wh"]
df.loc[df["energy_diff"] < 0, "energy_adjusted"] = np.nan  # Mark resets

df["energy_adjusted"] = df["energy_adjusted"].ffill()  # Forward-fill resets
df["energy_corrected_diff"] = df["energy_adjusted"].diff()
df["energy_rate_mW"] = (df["energy_corrected_diff"] * 1000) / df["timestamp"].diff().dt.total_seconds()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="timestamp", y="energy_rate_mW", hue="node", linewidth=2)
plt.xlabel("Timestamp")
plt.ylabel("Energy Consumption Rate (mW/s)")
plt.title("Rate of Energy Consumption Over Time (Corrected)")
plt.xticks(rotation=45)
plt.legend(title="Node", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


## Summary & Findings

Based on the visualizations and statistical analysis, we can derive the following insights:

- The power consumption varies across different nodes and runs.
- The cumulative energy consumption follows an increasing trend over time.
- Voltage and current appear stable with minor fluctuations.

Further analysis could involve:
- Identifying periods of peak energy usage.
- Comparing nodes to find efficiency variations.
- Investigating external factors influencing power consumption.